In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from nltk.corpus import names, movie_reviews
import nltk
import numpy as np
import pandas as pd

1.Using Gensim, train a doc2vec model on the Brown Corpus. Try to classify documents from each category.



In [49]:
import gensim.downloader as api

In [50]:
!pip install nltk

In [51]:
from nltk.corpus import brown

In [52]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [53]:
brown_corpus = nltk.corpus.brown

In [54]:
doc_ids = brown_corpus.fileids()
categories = brown_corpus.categories()

In [55]:
import gensim
from sklearn.model_selection import train_test_split
from nltk.corpus import brown
import pandas as pd

def read_corpus(doc_ids, tokens_only=False):
    for doc_id, category in zip(doc_ids, categories):
        text = ' '.join(brown.words(doc_id))
        tokens = gensim.utils.simple_preprocess(text)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags and categories
            yield gensim.models.doc2vec.TaggedDocument(tokens, [doc_id, category])

# Get document IDs and categories from the Brown corpus
doc_ids = brown.fileids()
categories = brown.categories()
#
train_corpus = [fileids for category in categories for fileids in brown.fileids(categories = category)]

# Split train_corpus into train and test sets
train_corpus, test_corpus = train_test_split(train_corpus, test_size=0.2, random_state=42)

print("Number of documents in train set:", len(train_corpus))
print("Number of documents in test set:", len(test_corpus))
print(len(train_corpus))
print(train_corpus)
print(test_corpus)
print(len(test_corpus))


Number of documents in train set: 400
Number of documents in test set: 100
400
['cj15', 'cd03', 'cn20', 'cf08', 'cf18', 'cg28', 'cj67', 'cr04', 'cf17', 'ck02', 'ck07', 'ca37', 'cf21', 'cn26', 'cc17', 'cj47', 'cj13', 'cj03', 'cb14', 'cg14', 'ce31', 'ch17', 'cf06', 'ck23', 'cr06', 'cr02', 'ca31', 'ce14', 'cb23', 'cf15', 'cg03', 'cb10', 'cp06', 'cj37', 'ck10', 'cg29', 'ce03', 'cn25', 'cn18', 'cj31', 'cg38', 'ce19', 'cp15', 'cg66', 'cj19', 'cj32', 'cn24', 'ce33', 'cj27', 'ca40', 'cn06', 'cb13', 'cg17', 'cn17', 'cc15', 'cf43', 'cn04', 'ce29', 'ca19', 'cg32', 'cb07', 'cf04', 'ca42', 'cg01', 'cd07', 'cp07', 'cn27', 'cn08', 'cc06', 'cb05', 'cg09', 'ck27', 'cp25', 'cb15', 'cb11', 'ch16', 'cl11', 'ch22', 'ck14', 'cl07', 'ca04', 'ce06', 'ca18', 'cj41', 'cc07', 'ck11', 'cl03', 'cg39', 'ce21', 'ch09', 'cp29', 'cl13', 'ca14', 'cj38', 'cb06', 'cj14', 'ck15', 'cg64', 'ck22', 'cl05', 'cp03', 'cg55', 'cj11', 'ch06', 'ch04', 'ce10', 'cr03', 'cg46', 'cp14', 'cf44', 'cj16', 'cb16', 'cj76', 'cj65', 'cj21', 

In [56]:
model = gensim.models.Doc2Vec(vector_size=100,min_count=2,epochs=40)
tagged_docs = list(read_corpus(train_corpus))
model.build_vocab(tagged_docs)

In [57]:
model.train(tagged_docs,total_examples= model.corpus_count,epochs=model.epochs)

In [58]:
def documents(model,test_corpus):
  for doc_id in test_corpus:
    words = brown.words(fileids = doc_id)
    test_tokens = gensim.utils.simple_preprocess(' '.join(words))
    i_vector = model.infer_vector(test_tokens)
    similar_docs = model.docvecs.most_similar([i_vector],topn=3)
    print(f"Document :{doc_id},Category : {brown.categories(fileids = doc_id)[0]}")
    print(f"Most similar docs:")
    for sim_doc_id,similarity in similar_docs:
      print(f"Document_id:{sim_doc_id},Similarity :{similarity}")
documents(model,test_corpus)

<ipython-input-58-357c81c8e77a>:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_docs = model.docvecs.most_similar([i_vector],topn=3)


Document :cf47,Category : lore
Most similar docs:
Document_id:belles_lettres,Similarity :0.7069841623306274
Document_id:cd03,Similarity :0.7067169547080994
Document_id:cr04,Similarity :0.5643598437309265
Document :cg45,Category : belles_lettres
Most similar docs:
Document_id:religion,Similarity :0.7111846804618835
Document_id:ca37,Similarity :0.7110491991043091
Document_id:cd03,Similarity :0.4717333912849426
Document :cl12,Category : mystery
Most similar docs:
Document_id:news,Similarity :0.6886856555938721
Document_id:ck07,Similarity :0.6884686946868896
Document_id:cr04,Similarity :0.6603367924690247
Document :ck25,Category : fiction
Most similar docs:
Document_id:cr04,Similarity :0.6693037152290344
Document_id:learned,Similarity :0.6687684059143066
Document_id:editorial,Similarity :0.6045268774032593
Document :cb01,Category : editorial
Most similar docs:
Document_id:religion,Similarity :0.7525434494018555
Document_id:ca37,Similarity :0.7524735331535339
Document_id:cd03,Similarity :0.

In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Use the stop word removal code from earlier on the 20 user groups:

• How does that effect the word mover distance of documents? Pick 6 documents to compare (make sure to use the same splits so they are the same document).

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
twe_news_df = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

# Tokenize texts and remove stop words
stop_words = set(stopwords.words('english'))
filtered_texts = [' '.join([word for word in word_tokenize(text.lower()) if word.isalpha() and word not in stop_words]) for text in twe_news_df.data]
stop_words_list = list(stop_words)

vectorizer_with_stopwords = CountVectorizer(stop_words=stop_words_list)
X_train_with_stopwords = vectorizer_with_stopwords.fit_transform(filtered_texts)

In [62]:
X_train_with_stopwords, X_test_with_stopwords, y_train, y_test = train_test_split(X_train_with_stopwords, twe_news_df.target, test_size=0.2, random_state=42)

log_reg_with_stopwords = LogisticRegression(max_iter=1000)    # Training logistic regression model with stop words removal
log_reg_with_stopwords.fit(X_train_with_stopwords, y_train)

y_train_pred_with_stopwords = log_reg_with_stopwords.predict(X_train_with_stopwords)
y_test_pred_with_stopwords = log_reg_with_stopwords.predict(X_test_with_stopwords)

In [63]:
# accuracy with stop words removal
train_accuracy_with_stopwords = accuracy_score(y_train, y_train_pred_with_stopwords)
test_accuracy_with_stopwords = accuracy_score(y_test, y_test_pred_with_stopwords)

print("Logistic Regression Accuracy with Stop Words Removal:")
print("Training Accuracy:", train_accuracy_with_stopwords)
print("Testing Accuracy:", test_accuracy_with_stopwords)

Logistic Regression Accuracy with Stop Words Removal:
Training Accuracy: 0.999779029941443
Testing Accuracy: 0.8789217852408308


Word model distance

How does the code show difference on logistic regression

In [64]:
pip install pot

In [65]:
documents_to_compare = [0, 10, 20, 30, 40, 50]  # Assuming these are indices of documents
tfidf_similarity = cosine_similarity(X_train_with_stopwords)

print("Cosine Similarity Comparison:")
for idx1 in documents_to_compare:
    for idx2 in documents_to_compare:
        similarity = tfidf_similarity[idx1, idx2]
        print(f"Similarity between Document {idx1 + 1} and Document {idx2 + 1}: {similarity}")

Cosine Similarity Comparison:
Similarity between Document 1 and Document 1: 0.9999999999999998
Similarity between Document 1 and Document 11: 0.06446025638903101
Similarity between Document 1 and Document 21: 0.03230104165360902
Similarity between Document 1 and Document 31: 0.05923488777590923
Similarity between Document 1 and Document 41: 0.0582716546748065
Similarity between Document 1 and Document 51: 0.02182987566053555
Similarity between Document 11 and Document 1: 0.06446025638903101
Similarity between Document 11 and Document 11: 1.0000000000000013
Similarity between Document 11 and Document 21: 0.0351649200941011
Similarity between Document 11 and Document 31: 0.03627381250550058
Similarity between Document 11 and Document 41: 0.04757860680698065
Similarity between Document 11 and Document 51: 0.03564803767780932
Similarity between Document 21 and Document 1: 0.03230104165360902
Similarity between Document 21 and Document 11: 0.0351649200941011
Similarity between Document 21 a

How does it effect the logistic regression classifier

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

def train_test_logistic_regression_without_stopwords(data, target, test_size=0.2, random_state=42):
    # Vectorize the text data without stop words removal
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(data)

    # Split the data into training and testing sets without stop words removal
    X_train, X_test, y_train, y_test = train_test_split(X_train, target, test_size=test_size, random_state=random_state)

    # Train logistic regression model without stop words removal
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)

    # Predict on testing set without stop words removal
    y_test_pred = log_reg.predict(X_test)

    # Calculate accuracy without stop words removal
    test_accuracy = accuracy_score(y_test, y_test_pred)

    return test_accuracy

# Fetch the 20 newsgroups dataset
twe_news_df = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

# Tokenize texts
filtered_texts = twe_news_df.data

# Train and test logistic regression model without stop words removal using the method
test_accuracy_without_stopwords = train_test_logistic_regression_without_stopwords(filtered_texts, twe_news_df.target)

# Print accuracy result
print("Logistic Regression Accuracy without Stop Words Removal:")
print("Testing Accuracy:", test_accuracy_without_stopwords)


Logistic Regression Accuracy without Stop Words Removal:
Testing Accuracy: 0.8806893504197967


So the removal of stop words does effect the accuracy of the  model and it is more accurate with stop word removal when compared to normal one

In [71]:
#function to remove the stop words
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    text_without_stop_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(text_without_stop_words)

In [72]:
# Removing the stop wording from train and test
X_train_without_stop_words = [remove_stop_words(text) for text in X_train]
X_test_without_stop_words = [remove_stop_words(text) for text in X_test]

In [73]:

tokenized_train_data = [word_tokenize(text) for text in X_train_without_stop_words]

In [74]:
from gensim.models import Word2Vec
import numpy as np

In [75]:
# Training Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_train_data, vector_size=300, window=5, min_count=1, workers=4)



In [76]:
# Defining function to calculate Word Mover's Distance (WMD) between documents
def calculate_wmd(doc1, doc2):
    doc1_tokens = word_tokenize(doc1)
    doc2_tokens = word_tokenize(doc2)
    return word2vec_model.wv.wmdistance(doc1_tokens, doc2_tokens)

Choosing six documents for finding mover distance

In [77]:
# Choosing six documents for comparison using indexes
doc_indices = [0, 10, 20, 30, 40, 50]
selected_docs_before = [X_test[i] for i in doc_indices]
selected_docs_after = [X_test_without_stop_words[i] for i in doc_indices]
# Calculating Word Mover's Distance for selected documents before and after stop word removal
wmd_before = []
wmd_after = []
for i in range(len(selected_docs_before)):
    wmd_before.append(calculate_wmd(selected_docs_before[i], X_train[doc_indices[i]]))
    wmd_after.append(calculate_wmd(selected_docs_after[i], X_train_without_stop_words[doc_indices[i]]))
average_wmd_before = np.mean(wmd_before)
average_wmd_after = np.mean(wmd_after)
print("Average WMD Before Stop Word Removal:", average_wmd_before)
print("Average WMD After Stop Word Removal:", average_wmd_after)

Average WMD Before Stop Word Removal: 0.6625145976924062
Average WMD After Stop Word Removal: 0.6723456872219623
